# MVP

In [19]:
import pandas as pd
import numpy as np

import logging
import boto3
from botocore.exceptions import ClientError

In [2]:
pd.read_json("df_ds_tx_prepared.json")

,title,company,company_rating,job_link,job_description,Unnamed: 0,city,state,zipcode,clean,tokenized,stemmed,lemmatized
0,CCOR Card Compliance Data Science/Analytics As...,"JPMorgan Chase Bank, N.A.",3.9,https://www.indeed.com/rc/clk?jk=52527d22aa9a4...,Consumer & Community Banking (CCB) Compliance ...,2.0,Plano,TX,0,consumer community banking ccb compliance team...,consumer community banking ccb compliance team...,consum commun bank ccb complianc team play a c...,consumer community banking ccb compliance team...
1,Senior Machine Learning R&D Engineer\nnew,Ultra,4.0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Ultra Labs is seeking a talented, highly motiv...",0.0,Austin,TX,0,ultra lab seeking talented highly motivated se...,ultra labs is seeking a talented highly motiva...,ultra lab is seek a talent highli motiv senior...,ultra lab is seeking a talented highly motivat...
2,Data Scientist Senior - Computer Vision/Deep L...,USAA,3.9,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job We are currently seeking a tale...,3.0,LaCoste,TX,78039,purpose job currently seeking talented data sc...,purpose of job we are currently seeking a tale...,purpos of job we are current seek a talent dat...,purpose of job we are currently seeking a tale...
3,Decision Science Analyst Lead - Personal Lines...,USAA,3.9,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job Provide decision support for bu...,4.0,LaCoste,TX,78039,purpose job provide decision support business ...,purpose of job provide decision support for bu...,purpos of job provid decis support for busi ar...,purpose of job provide decision support for bu...
4,Senior Data Scientist\nnew,Argus Media,3.4,https://www.indeed.com/rc/clk?jk=c35d5fedf4f40...,Job Title:Senior Data Scientist – AI/ML Quant\...,1.0,Houston,TX,0,job titlesenior data scientist aiml quant job ...,job titlesenior data scientist aiml quant\njob...,job titlesenior data scientist aiml quant job ...,job titlesenior data scientist aiml quant job ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,Data Scientist,Luminator Technology Group,4.1,https://www.indeed.com/rc/clk?jk=e64e6c9cdbfdc...,Data Scientist\nLuminator Technology Group\nPl...,NaN,Plano,TX,75074,data scientist luminator technology group plan...,data scientist\nluminator technology group\npl...,data scientist lumin technolog group plano tx ...,data scientist luminator technology group plan...
1111,Data Scientist / Data Engineer,SEVEN,3.8,https://www.indeed.com/rc/clk?jk=ae2fad598ffe3...,Job Description\nSEVEN Networks develops innov...,NaN,Marshall,TX,75672,job description seven network develops innovat...,job description\nseven networks develops innov...,job descript seven network develop innov mobil...,job description seven network develops innovat...
1112,Data Scientist I,AIG,3.7,https://www.indeed.com/rc/clk?jk=dca2d29109068...,Data Scientist I - NLP\nWho we are\nAmerican I...,NaN,Houston,TX,0,data scientist nlp american international grou...,data scientist i nlp\nwho we are\namerican int...,data scientist i nlp who we are american inter...,data scientist i nlp who we are american inter...
1113,Associate Data Scientist,BroadVail Capital Partners,0.0,https://www.indeed.com/company/BroadVail-Capit...,BroadVail Capital Partners is a commercial rea...,NaN,Houston,TX,77056,broadvail capital partner commercial real esta...,broadvail capital partners is a commercial rea...,broadvail capit partner is a commerci real est...,broadvail capital partner is a commercial real...


In [9]:
df = pd.read_csv("ds_tx_indeed_02012021.csv")
df = df.drop(columns=['Unnamed: 0'])
df.to_csv("ds_tx_indeed_02012021.csv", index=False)

In [12]:
pd.read_csv("ds_tx_indeed_02012021.csv")

,title,location,company,company_rating,post_age,job_link,job_description
0,Senior Machine Learning R&D Engineer\nnew,"Austin, TX",Ultra,4.0,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Ultra Labs is seeking a talented, highly motiv..."
1,Senior Data Scientist\nnew,"Houston, TX",Argus Media,3.4,Just posted,https://www.indeed.com/rc/clk?jk=c35d5fedf4f40...,Job Title:Senior Data Scientist – AI/ML Quant\...
2,CCOR Card Compliance Data Science/Analytics As...,"Plano, TX","JPMorgan Chase Bank, N.A.",3.9,Today,https://www.indeed.com/rc/clk?jk=52527d22aa9a4...,Consumer & Community Banking (CCB) Compliance ...
3,Data Scientist Senior - Computer Vision/Deep L...,"LaCoste, TX 78039",USAA,3.9,Today,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job We are currently seeking a tale...
4,Decision Science Analyst Lead - Personal Lines...,"LaCoste, TX 78039",USAA,3.9,Today,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job Provide decision support for bu...
...,...,...,...,...,...,...,...
520,DPS - FIN - CAPPS Financial Senior Data Analyst,"Austin, TX 78752 (Highland area)",TEXAS DEPARTMENT OF PUBLIC SAFETY,3.8,30+ days ago,https://www.indeed.com/rc/clk?jk=5d31664266a09...,DPS - FIN - CAPPS Financial Senior Data Analys...
521,Senior AI Engineer,"Austin, TX 78759 (Arboretum area)",ZeniMax Media Inc.,4.0,30+ days ago,https://www.indeed.com/rc/clk?jk=6f29bf232adbd...,Overview:\n\nArkane Studios is looking for a S...
522,Senior AI Engineer,"Austin, TX",IllFonic,4.5,30+ days ago,https://www.indeed.com/rc/clk?jk=1b3a186505159...,Senior AI Programmer\nAbout the Job\nIllFonic ...
523,Data Scientist Machine Learning Engineer,"Dallas, TX",Plasma Computing Group,4.0,30+ days ago,https://www.indeed.com/rc/clk?jk=8141efa09435e...,"Primary Responsibilities to Include:\nDesign, ..."


In [15]:
df1 = pd.read_json("df_ds_tx_prepared.json")
df1 = df1.drop(columns=["Unnamed: 0"])
df1.to_json("df_ds_tx_prepared.json")

In [16]:
pd.read_json("df_ds_tx_prepared.json")

,title,company,company_rating,job_link,job_description,city,state,zipcode,clean,tokenized,stemmed,lemmatized
0,CCOR Card Compliance Data Science/Analytics As...,"JPMorgan Chase Bank, N.A.",3.9,https://www.indeed.com/rc/clk?jk=52527d22aa9a4...,Consumer & Community Banking (CCB) Compliance ...,Plano,TX,0,consumer community banking ccb compliance team...,consumer community banking ccb compliance team...,consum commun bank ccb complianc team play a c...,consumer community banking ccb compliance team...
1,Senior Machine Learning R&D Engineer\nnew,Ultra,4.0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Ultra Labs is seeking a talented, highly motiv...",Austin,TX,0,ultra lab seeking talented highly motivated se...,ultra labs is seeking a talented highly motiva...,ultra lab is seek a talent highli motiv senior...,ultra lab is seeking a talented highly motivat...
2,Data Scientist Senior - Computer Vision/Deep L...,USAA,3.9,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job We are currently seeking a tale...,LaCoste,TX,78039,purpose job currently seeking talented data sc...,purpose of job we are currently seeking a tale...,purpos of job we are current seek a talent dat...,purpose of job we are currently seeking a tale...
3,Decision Science Analyst Lead - Personal Lines...,USAA,3.9,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Purpose of Job Provide decision support for bu...,LaCoste,TX,78039,purpose job provide decision support business ...,purpose of job provide decision support for bu...,purpos of job provid decis support for busi ar...,purpose of job provide decision support for bu...
4,Senior Data Scientist\nnew,Argus Media,3.4,https://www.indeed.com/rc/clk?jk=c35d5fedf4f40...,Job Title:Senior Data Scientist – AI/ML Quant\...,Houston,TX,0,job titlesenior data scientist aiml quant job ...,job titlesenior data scientist aiml quant\njob...,job titlesenior data scientist aiml quant job ...,job titlesenior data scientist aiml quant job ...
...,...,...,...,...,...,...,...,...,...,...,...,...
1110,Data Scientist,Luminator Technology Group,4.1,https://www.indeed.com/rc/clk?jk=e64e6c9cdbfdc...,Data Scientist\nLuminator Technology Group\nPl...,Plano,TX,75074,data scientist luminator technology group plan...,data scientist\nluminator technology group\npl...,data scientist lumin technolog group plano tx ...,data scientist luminator technology group plan...
1111,Data Scientist / Data Engineer,SEVEN,3.8,https://www.indeed.com/rc/clk?jk=ae2fad598ffe3...,Job Description\nSEVEN Networks develops innov...,Marshall,TX,75672,job description seven network develops innovat...,job description\nseven networks develops innov...,job descript seven network develop innov mobil...,job description seven network develops innovat...
1112,Data Scientist I,AIG,3.7,https://www.indeed.com/rc/clk?jk=dca2d29109068...,Data Scientist I - NLP\nWho we are\nAmerican I...,Houston,TX,0,data scientist nlp american international grou...,data scientist i nlp\nwho we are\namerican int...,data scientist i nlp who we are american inter...,data scientist i nlp who we are american inter...
1113,Associate Data Scientist,BroadVail Capital Partners,0.0,https://www.indeed.com/company/BroadVail-Capit...,BroadVail Capital Partners is a commercial rea...,Houston,TX,77056,broadvail capital partner commercial real esta...,broadvail capital partners is a commercial rea...,broadvail capit partner is a commerci real est...,broadvail capital partner is a commercial real...


In [17]:
def upload_to_S3_bucket(file_name, bucket='dspreparedjobpostings', object_name=None):
    """
    Upload a file to an S3 bucket
    
    ***Prepared data files must be in JSON format***

    Parameters
    ----------
    file_name: str
        Name of the file to upload.
    
    bucket: str, default="dspreparedjobpostings"
        S3 Bucket the file will be uploaded to.
    
    object_name: str, default=None
        The file name that will appear in AWS S3 bucket.
        If an object_name is not specified, the file will
        have the same name as the file_name
    
    Returns
    -------
    True or False: bool
        True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [20]:
upload_to_S3_bucket("df_ds_tx_prepared.json")

True